# 岛屿检测准确率评估
读取标注 JSON 与检测图像，评估每对图片的像素级重合率（IoU），并绘制准确率对比图。

In [ ]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 设置测试样本数量及初始化变量

In [ ]:
n = 10  # 测试集中图片对数
accuracies = []  # 存储每对图像的IoU

## 主循环：读取每对图像与JSON并评估IoU

In [ ]:
for i in range(1, n+1):
    json_path = fr"E:/sophomore/study/pytorch/test/test{i}.json"
    image_path = fr"E:/sophomore/study/pytorch/test/test{i}_image.jpg"

    with open(json_path, "r") as f:
        json_data = json.load(f)
    saved_image = cv2.imread(image_path)

    image_width = json_data["imageWidth"]
    image_height = json_data["imageHeight"]
    island_mask = np.zeros((image_height, image_width), dtype=np.uint8)
    for shape in json_data["shapes"]:
        if shape["label"] == "island":
            points = np.array(shape["points"], dtype=np.int32)
            cv2.fillPoly(island_mask, [points], 255)
    resized_mask = cv2.resize(island_mask, (450, 300), interpolation=cv2.INTER_NEAREST)
    saved_image_resized = cv2.resize(saved_image, (450, 300), interpolation=cv2.INTER_NEAREST)

    hsv_image = cv2.cvtColor(saved_image_resized, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 50, 50])
    upper_green = np.array([85, 255, 255])
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])
    red_mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    red_mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(red_mask1, red_mask2)
    detected_island_mask = cv2.bitwise_or(green_mask, red_mask)

    intersection = np.logical_and(resized_mask, detected_island_mask).sum()
    union = np.logical_or(resized_mask, detected_island_mask).sum()
    accuracy = intersection / union if union > 0 else 0
    print(f"{i}:", accuracy)
    accuracies.append(accuracy)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(resized_mask, cmap="gray")
    plt.axis('off')
    plt.title("GT Mask")
    plt.subplot(1, 2, 2)
    plt.imshow(saved_image_resized[:, :, ::-1])
    plt.axis('off')
    plt.title("Predicted Image")
    plt.show()

## 准确率图表可视化（柱状图 + 折线图）

In [ ]:
x = [str(i+1) for i in range(n)]
y = accuracies
min_prob = min(y)
min_index = x[y.index(min_prob)]

plt.figure(figsize=(8, 8))
plt.bar(x, y, color='skyblue', alpha=0.6, label="Probability (Bar)")
plt.plot(x, y, marker='o', linestyle='-', color='b', label="Probability (Line)")
plt.axhline(y=min_prob, color='r', linestyle='--', linewidth=1)
plt.text(0.1, min_prob, f"{min_prob:.3f}", color='r', verticalalignment='bottom')
plt.title("Probability Distribution (Bar & Line)")
plt.xlabel("Index")
plt.ylabel("Probability")
plt.ylim(0, 1.0)
plt.xticks(x)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()